<a href="https://colab.research.google.com/github/GiamTeo/Serie_A_Analysis/blob/main/interfaccia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from collections import Counter
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


# Carica il file CSV
# uploaded = files.upload()

# Leggi il file CSV
for file_name in uploaded.keys():
    data = pd.read_csv(file_name)

# Definizione delle dimensioni del campo da calcio
field_length = 120
field_width = 80

def show_results(data, squadra):
    # Filtra i dati per la squadra specificata
    squadra_data = data[(data['team_name_home'] == squadra) | (data['team_name_away'] == squadra)].copy()
    # Estrai il numero della giornata dalla colonna 'stage'
    squadra_data['matchday'] = squadra_data['stage'].apply(extract_matchday)
    # Ordina i dati per il numero della giornata in ordine crescente
    squadra_sorted = squadra_data.sort_values(by='matchday', ascending=True)
    # Seleziona solo le colonne pertinenti
    risultati = squadra_sorted[['date', 'stage', 'team_name_home', 'team_home_score', 'team_away_score', 'team_name_away']]
    # Visualizza i risultati
    display(risultati)

def extract_matchday(stage):
    return int(stage.split()[1])

# Funzione per visualizzare la formazione più volte schierata di una squadra
def most_common_lineup(data, squadra):
    posizioni = {
        'ST': (100,40),
        'LW': (80,70),
        'RW': (80,10),
        'LM': (65,50),
        'CM': (65,40),
        'RM': (65,30),
        'LB': (40,70),
        'CB1': (40,45),
        'CB2': (40,35),
        'RB': (40,10),
        'GK': (20,40)
    }

    # Filtra i dati per la squadra specificata
    squadra_data = data[(data['team_name_home'] == squadra) | (data['team_name_away'] == squadra)]

    # Estrarre la formazione più comune
    formazione_più_comune = Counter(squadra_data['player_names_home'].tolist() + squadra_data['player_names_away'].tolist()).most_common(1)[0][0]
    giocatori = formazione_più_comune.split(',')

    # Creare un dizionario per memorizzare i giocatori per ogni ruolo
    giocatori_per_ruolo = {ruolo: [] for ruolo in posizioni.keys()}

    # Associa i giocatori ai ruoli
    for ruolo, giocatore in zip(posizioni.keys(), giocatori):
        giocatori_per_ruolo[ruolo].append(giocatore)

    # Disegna il campo da calcio
    plt.figure(figsize=(60, 40), edgecolor = 'green', facecolor = 'green')
    plt.plot([0, field_length], [0, 0], color='green')
    plt.plot([0, field_length], [field_width, field_width], color='green')
    plt.plot([0, 0], [0, field_width], color='green')
    plt.plot([field_length, field_length], [0, field_width], color='green')

    # Disegna i giocatori presenti sul campo
    for ruolo, giocatori in giocatori_per_ruolo.items():
        for giocatore in giocatori:
            posizione = posizioni.get(ruolo, None)
            if posizione:
                plt.gca().add_patch(Rectangle((posizione[0]-5, posizione[1]-4), 10, 4, color='yellow'))
                text_height = 4  # altezza rettangolo testo
                plt.text(posizione[0], posizione[1] - text_height/2, giocatore, ha='center', va='center', fontsize=40, color='black')

    plt.xlim(0, field_length)
    plt.ylim(0, field_width)
    plt.axis('off')
    plt.title('Formazione più comune per ' + squadra)
    plt.show()

# Funzione per calcolare le statistiche sui gol di una squadra
def goal_statistics(data, squadra):
    squadra_home = data[data['team_name_home'] == squadra]
    squadra_away = data[data['team_name_away'] == squadra]

    gol_segnati = squadra_home['team_home_score'].sum() + squadra_away['team_away_score'].sum()
    gol_subiti = squadra_home['team_away_score'].sum() + squadra_away['team_home_score'].sum()

    print("Gol segnati da", squadra, ":", gol_segnati)
    print("Gol subiti da", squadra, ":", gol_subiti)

# Funzione per visualizzare la classifica finale
def final_standings(data, squadra_selected):
    squadre = data['team_name_home'].unique()
    classifica = pd.DataFrame()

    for squadra in squadre:
        squadra_home = data[data['team_name_home'] == squadra]
        squadra_away = data[data['team_name_away'] == squadra]

        punti = ((squadra_home['team_home_score'] > squadra_home['team_away_score']).sum() +
                 (squadra_away['team_away_score'] > squadra_away['team_home_score']).sum()) * 3 + \
                ((squadra_home['team_home_score'] == squadra_home['team_away_score']) |
                 (squadra_away['team_away_score'] == squadra_away['team_home_score'])).sum()

        classifica = pd.concat([classifica, pd.DataFrame({'Squadra': [squadra], 'Punti': [punti]})], ignore_index=True)

    classifica = classifica.sort_values(by='Punti', ascending=False).reset_index(drop=True)
    # Evidenzia la riga corrispondente alla squadra selezionata
    classifica_styled = classifica.style.apply(lambda row: ['background: black' if row['Squadra'] == squadra_selected else '' for idx, value in row.items()], axis=1)
    display(classifica_styled)

def total_shots(data, squadra):
    squadra_home = data[data['team_name_home'] == squadra]
    squadra_away = data[data['team_name_away'] == squadra]
    # Calcola i tiri totali stagionali della squadra selezionata
    tiri_totali = squadra_home['total_shots_home'].sum() + squadra_away['total_shots_away'].sum()
    print(f"I tiri totali della squadra {squadra} sono: {tiri_totali}")

def possession_average(data, squadra):
    squadra_home = data[data['team_name_home'] == squadra]
    squadra_away = data[data['team_name_away'] == squadra]
    # Calcola la media del possesso palla stagionale
    possesso_palla = (squadra_home['possession_home'].mean() + squadra_away['possession_away'].mean()) / 2
    possesso_palla = round(possesso_palla, 5)
    print(f"La media del possesso palla della squadra {squadra} è {possesso_palla*100} %")

squadra_dropdown = widgets.Dropdown(options=data['team_name_home'].unique(), description='Scegli una squadra:')
button_results = widgets.Button(description='Mostra risultati')
button_lineup = widgets.Button(description='Mostra formazione più comune')
button_goals = widgets.Button(description='Mostra statistiche sui gol')
button_standings = widgets.Button(description='Mostra classifica finale')
button_shots = widgets.Button(description='Mostra tiri totali')
button_possession = widgets.Button(description='Mostra media possesso palla')
output = widgets.Output()

# Definizione delle azioni dei bottoni
def show_results_action(_):
    output.clear_output()
    with output:
        show_results(data, squadra_dropdown.value)

def lineup_action(_):
    output.clear_output()
    with output:
        most_common_lineup(data, squadra_dropdown.value)

def goals_action(_):
    output.clear_output()
    with output:
        goal_statistics(data, squadra_dropdown.value)

def standings_action(_):
    output.clear_output()
    with output:
        final_standings(data, squadra_dropdown.value)

def shots_action(_):
    output.clear_output()
    with output:
        total_shots(data, squadra_dropdown.value)

def possession_action(_):
    output.clear_output()
    with output:
        possession_average(data, squadra_dropdown.value)

button_results.on_click(show_results_action)
button_lineup.on_click(lineup_action)
button_goals.on_click(goals_action)
button_standings.on_click(standings_action)
button_shots.on_click(shots_action)
button_possession.on_click(possession_action)

button_results.layout.width = '200px'
button_lineup.layout.width = '200px'
button_goals.layout.width = '200px'
button_standings.layout.width = '200px'
button_shots.layout.width = '200px'
button_possession.layout.width = '200px'

# Visualizzazione dei widget
display(squadra_dropdown)
display(button_results)
display(button_lineup)
display(button_goals)
display(button_standings)
display(button_shots)
display(button_possession)
display(output)